In [ ]:
!pip install transformers
import pandas as pd
import numpy as np
import torch
import tensorflow as tf
from google.colab import drive 
drive.mount('/content/drive')
from transformers import AutoTokenizer,AutoModel
from torch.utils.data import TensorDataset
from tqdm import tqdm
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=True)
model = AutoModel.from_pretrained('bert-base-uncased').eval()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# import re

# def remove_special_chars(text):

#   clean_text = re.sub(r"[^\w\s.,]", "", text)
#   return re.sub(r"\b\w\b", "", clean_text)
# from tqdm import tqdm

# def limit_string_length(string_list, max_word_count=500):
#     """限制字符串列表中每个字符串的长度"""
#     for i, string in tqdm(enumerate(string_list), total=len(string_list)):
#         string = remove_special_chars(string)
#         words = string.split()[:max_word_count]
#         string_list[i] = " ".join(words)
#     return string_list


In [ ]:
a = torch.load('/content/drive/MyDrive/data/token/open_token1.pt')

In [ ]:
from tqdm import tqdm
import math

def embedding(input_ids, batch_size=100):
    input_ids.cpu()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    with torch.no_grad():
        num_parts = 50#将input_ids 平均分成50份(可调整),每处理完一份保存到硬盘文件夹然后清空RAM
        part_size = math.ceil(len(input_ids) / num_parts)#例如总共95806条文本,每份95806/50= 1916条文本，一份有1916/100个batch
        num_batches = math.ceil(part_size / batch_size)

        for i in range(num_parts):
            start = i * part_size
            end = min((i + 1) * part_size, len(input_ids))
            part_embeddings = []

            # 每份的batch szie
            for j in tqdm(range(num_batches)):
                batch_start = start + j * batch_size
                batch_end = min(start + (j + 1) * batch_size, end)

                input_ids_batch = input_ids[batch_start:batch_end].to(device)

                outputs = model(input_ids_batch, output_hidden_states=True)
                hidden_states = outputs[2]

                token_embeddings_batch = torch.stack(hidden_states, dim=0)
                tem = torch.sum(token_embeddings_batch[-4:], dim=0).cpu()
                part_embeddings.append(tem)
                del token_embeddings_batch, hidden_states,outputs,input_ids_batch
                torch.cuda.empty_cache()

            part_embeddings = torch.cat(part_embeddings, dim=0)
            #路径根据处理的数据集改一下，放到一个文件夹里
            torch.save(part_embeddings, f'openoffice_b2_part_{i}.pt')

            del part_embeddings
            torch.cuda.empty_cache()
#最终会在文件夹中保存50个.pt的张量文件，之后按顺序批量读取合并成一个就好了，一定要按顺序要不和标签啥的不匹配了
#每个数据集有两个token文件，例如eclipse_token1.pt和eclipse_token1.pt。这两个文件都需要处理，分文件夹保存之后读取
#eclipse_token1生成的50个张量文件合成一个新张量，eclipse_token2生成的50个张量文件合成另一个新张量。按照此类方法处理三个数据集
#上述的part_size = 50 仅为示例，可自行调整。
    return 0


In [ ]:
 c = embedding(a[:111])#用111条文本测试了一下，能够正常运行

100%|██████████| 1/1 [00:00<00:00, 60.88it/s]
